# Pandas確認課題

このPandas確認問題は、データサイエンス100本ノックの問題で最低限必要な問題を抜粋したものですが、[Introduction_to_Pandas](./11_Introduction_to_Pandas.ipynb) に掲載されていない機能が必要な問題もあります。
初めて触るライブラリを調べながら使うというのはよくある光景です。この課題では皆さんにもそれに挑戦していただきます。  
ヒントとして検索キーワードなどを載せておくので、自力で調べながら解いてみましょう。  



## 必要モジュールのインポート

この問題で使うモジュールをインポートします．

In [1]:
import pandas as pd
import numpy as np

## データの読み込み

In [2]:
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

---
## 問1. 条件抽出
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [17]:
print(df_receipt.query('customer_id == "CS018205000001" or (amount >= 1000 or quantity >= 5)'))

        sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
17       20181225   1545696000   S13020        1172               2   
36       20180911   1536624000   S13018        1122               2   
40       20170501   1493596800   S13004        1192               2   
73       20170523   1495497600   S14028        1182               2   
75       20180310   1520640000   S14028         112               1   
...           ...          ...      ...         ...             ...   
104622   20170801   1501545600   S13017        1102               2   
104656   20170730   1501372800   S13005        1102               2   
104662   20191024   1571875200   S14011        1132               2   
104666   20170330   1490832000   S13017        1112               1   
104669   20190726   1564099200   S14006        1122               1   

           customer_id  product_cd  quantity  amount  
17      ZZ000000000000  P071401002         1    1100  
36      CS018205000001  P071401012   

---
## 問2. ソート
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭5件を全項目表示せよ。

In [4]:
print(df_customer.sort_values(by='birth_day', ascending=False).head(5))

          customer_id customer_name  gender_cd gender   birth_day  age  \
15639  CS035114000004         大村 美里          1     女性  2007-11-25   11   
7468   CS022103000002        福山 はじめ          9     不明  2007-10-02   11   
10745  CS002113000009        柴田 真悠子          1     女性  2007-09-17   11   
19811  CS004115000014         松井 京子          1     女性  2007-08-09   11   
7039   CS002114000010          山内 遥          1     女性  2007-06-03   11   

      postal_cd                address application_store_cd  application_date  \
15639  156-0053     東京都世田谷区桜**********               S13035          20150619   
7468   249-0006    神奈川県逗子市逗子**********               S14022          20160909   
10745  184-0014  東京都小金井市貫井南町**********               S13002          20160304   
19811  165-0031    東京都中野区上鷺宮**********               S13004          20161120   
7039   184-0015  東京都小金井市貫井北町**********               S13002          20160920   

          status_cd  
15639  6-20091205-6  
7468   0-00000000-0  
10

---
## 問3. 全件数
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [5]:
print(len(df_receipt))

104681


## 問4. ユニーク件数
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [6]:
print(df_receipt['customer_id'].nunique())

8307


<details>
<summary>ヒント</summary>
「ユニーク」というのはそのまま検索に使える単語です。  
</details>

---
## 問5. 〇〇ごとに集計
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは先頭5件だけ表示せよ。

会員のみを抽出する方法は、例えば以下の2通りの方法があります。

In [7]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
df_receipt_only_member = df_receipt.query("not customer_id.str.startswith('Z')", engine="python")

In [8]:
df_grouped = df_receipt_only_member.groupby('customer_id').sum()
ave_amount = df_grouped.mean(numeric_only=True)['amount']
print(df_grouped.query('amount >= @ave_amount').head(5)['amount'])

customer_id
CS001115000010    3044
CS001205000006    3337
CS001214000009    4685
CS001214000017    4132
CS001214000052    5639
Name: amount, dtype: int64


<details>
<summary>ヒント1</summary>
「pandas 要素ごと 集計」 などで今回使える機能に関する記事が見つかります。
</details>

<details>
<summary>ヒント2</summary>
メソッド名は "groupby" です。
</details>

---
## 問6. DataFrameの結合
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は先頭5件だけ表示せよ。

In [9]:
df_customer_only_member = df_customer[~df_customer["customer_id"].str.startswith("Z")]
df_customer_only_member = df_customer.query("not customer_id.str.startswith('Z')", engine="python")

In [20]:
df_receipt_grouped = df_receipt_only_member.groupby('customer_id').sum().drop(columns=['sales_ymd', 'sales_epoch', 'store_cd', 'receipt_no', 'receipt_sub_no', 'product_cd', 'quantity'])
print(pd.merge(df_customer_only_member.set_index('customer_id'), df_receipt_grouped, on= 'customer_id', how='left').fillna({'amount': 0}).head(5))

               customer_name  gender_cd gender   birth_day  age postal_cd                 address application_store_cd  application_date     status_cd  amount
customer_id                                                                                                                                                   
CS021313000114        大野 あや子          1     女性  1981-04-29   37  259-1113    神奈川県伊勢原市粟窪**********               S14021          20150905  0-00000000-0     0.0
CS037613000071         六角 雅彦          9     不明  1952-04-01   66  136-0076      東京都江東区南砂**********               S13037          20150414  0-00000000-0     0.0
CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42  151-0053     東京都渋谷区代々木**********               S13031          20150529  D-20100325-C  5088.0
CS028811000001        堀井 かおり          1     女性  1933-03-27   86  245-0016  神奈川県横浜市泉区和泉町**********               S14028          20160115  0-00000000-0     0.0
CS001215000145         田崎 美紀          1     女性

<details>
<summary>ヒント1</summary>
タイトル通り 「pandas DataFrame 結合」などと調べれば必要な機能に関する記事が見つかります。  
</details>


<details>
<summary>ヒント2</summary>
"merge", "join"という似たメソッドがあります。  
今回の場合"merge"が便利でしょう。
</details>

---
## 問7. 時系列データ
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。なお、データは先頭5件を表示せよ。

In [11]:
df_customer_applicationdate = df_customer.set_index('customer_id')
print(pd.to_datetime(df_customer_applicationdate['application_date'], format="%Y%m%d").head(5))

customer_id
CS021313000114   2015-09-05
CS037613000071   2015-04-14
CS031415000172   2015-05-29
CS028811000001   2016-01-15
CS001215000145   2017-06-05
Name: application_date, dtype: datetime64[ns]


<details>
<summary>ヒント1</summary>
「pandas datetime」などで該当の機能が見つかるかと思います。
</details>


<details>
<summary>ヒント2</summary>
"pd.to_datetime"というメソッドが使えるでしょう。このメソッドを適用する際ですが、for文を使わずに実装しましょう。

---
## 問8. 関数
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、結果は先頭5件を表示せよ。

In [12]:
df_amount_ln = df_grouped.drop(columns=['sales_ymd', 'sales_epoch', 'store_cd', 'receipt_no', 'receipt_sub_no', 'product_cd', 'quantity'])
df_amount_ln['log_amount'] = np.log10(df_amount_ln['amount'])
print(df_amount_ln)

                amount  log_amount
customer_id                       
CS001113000004    1298    3.113275
CS001114000005     626    2.796574
CS001115000010    3044    3.483445
CS001205000004    1988    3.298416
CS001205000006    3337    3.523356
...                ...         ...
CS051212000001     336    2.526339
CS051513000004     551    2.741152
CS051515000002     265    2.423246
CS052212000002     192    2.283301
CS052514000001     178    2.250420

[8306 rows x 2 columns]


---
## 問9. 欠損数
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [13]:
print(df_product.isnull().sum())

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64


---
## 問10. 欠損値の除去
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [14]:
df_product_1 = df_product.dropna(how='any', axis=0)

In [15]:
print(len(df_product), len(df_product_1))

10030 10023


---
## 問11. 欠損値の穴埋め
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とせよ。補完実施後、各項目について欠損が生じていないことも確認すること。

In [16]:
ave_price = df_product.mean(numeric_only=True)['unit_price']
ave_cost = df_product.mean(numeric_only=True)['unit_cost']
df_product_2 = df_product.fillna({'unit_price': ave_price, 'unit_cost': ave_cost})
print(df_product_2.isnull().sum())

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64


### 余談
ChatGPTやBing AIに聞けば大抵のことは教えてくれます。  
何回か入力文章を吟味しないといけないこともありますが、知らないことを調べる場合は自分で検索するよりも早いです。  
ただ、ChatGPTなどは嘘をつく場合があるので、自分でソースを参照する姿勢は必要です。  

これはBingAIの回答例です。  

![BingAIの回答例](./imgs/pandas/BingAI.png)